In [3]:
import pyopencl as cl

# Configuramos el contexto y cola de comandos
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [4]:
import numpy as np

# Obtenemos el tamaño máximo de grupo de trabajo y el tamaño máximo de los
# elementos de grupo de trabajo en cada dimensión
max_work_group_size = device.get_info(cl.device_info.MAX_WORK_GROUP_SIZE)
max_work_item_sizes = device.get_info(cl.device_info.MAX_WORK_ITEM_SIZES)
print(
    f"Máximo tamaño de grupo: {max_work_group_size}, " 
    f"máximo tamaño de elementos por dimensión: {max_work_item_sizes}")

# Definimos el tamaño de los bloques
block_size = int(np.sqrt(max_work_group_size))
block_size_x = min(block_size, max_work_item_sizes[0])
block_size_y = min(block_size, max_work_item_sizes[1])
print(f"Elementos por bloque: {block_size_x} x {block_size_y}")

Máximo tamaño de grupo: 1024, máximo tamaño de elementos por dimensión: [1024, 1024, 64]
Elementos por bloque: 32 x 32


In [5]:
# Definimos el tamaño de las matrices A(n x l) y B (l x m)
m = 4000
l = 6000
n = 2000
np.random.seed(13)
a_cpu = np.random.rand(m, l).astype(np.float32)
b_cpu = np.random.rand(l, n).astype(np.float32)
c_cpu = np.empty((m, n), dtype=np.float32)

# Creamos buffers en el dispositivo
mf = cl.mem_flags
a_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_cpu)
b_buf = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_cpu)
c_buf = cl.Buffer(context, mf.WRITE_ONLY, c_cpu.nbytes)

In [6]:
# Definimos el kenel OpenCL
program_source = """
__kernel void matmul(const int M, const int N, const int K,
                     __global const float *A, __global const float *B,
                     __global float *C) {
    int row = get_global_id(0);
    int col = get_global_id(1);
    
    float sum = 0.0f;
    for (int k = 0; k < K; k++) {
        sum += A[row * K + k] * B[k * N + col];
    }
    C[row * N + col] = sum;
}
"""
program = cl.Program(context, program_source).build()

In [7]:
# Ajustamos el tamaño global (grilla) 
global_size_x = (m + block_size_x - 1) // block_size_x * block_size_x
global_size_y = (n + block_size_y - 1) // block_size_y * block_size_y
global_size = (global_size_x, global_size_y)
local_size = (block_size_x, block_size_y)

# Ejecutamos el kernel
matmul_kernel = program.matmul
matmul_kernel.set_args(np.int32(m), np.int32(n), np.int32(l), a_buf, b_buf, c_buf)
cl.enqueue_nd_range_kernel(queue, matmul_kernel, global_size, local_size)

# Transferimos el resultado al host
cl.enqueue_copy(queue, c_cpu, c_buf)

# Esperamos a que terminen las operaciones
queue.finish()

In [8]:
# Calculamos el resultado en el host para comparar con el del device
c_npy = np.dot(a_cpu, b_cpu)

# Comparamos los resultados obtenidos en la CPU y la GPU
np.allclose(c_npy, c_cpu, rtol=1e-01, atol=1e-8)

True


### Copyright 2020-2024 Facundo Batista y Manuel Carlevaro

Licencia CC BY-NC-SA 4.0

Para más info visitar: https://github.com/facundobatista/libro-pyciencia/

